In [ ]:
import json

path = "ALQAC_2025_data/alqac25_train.json"

with open(path, "r", encoding="utf-8") as f:
    json_data = json.load(f)

json_data[:2]

In [ ]:
import json

data_path = "ALQAC_2025_data/alqac25_law.json"
with open(data_path, "r", encoding="utf-8") as f:
    corpus = json.load(f)
corpus[0]

In [ ]:
import os
from openai import OpenAI


# model_name = "DeepSeek-R1-Distill-Llama-8B"
# model_name = "Qwen2.5-7B-Instruct"
model_name = "Llama-3.1-8B-Instruct"
# model_name = "llama-3.1-8b-instant"
from dotenv import load_dotenv

load_dotenv("llm-fpt/.env")
)


class FPTLlm:
    def __init__(self, model_name: str = "Qwen2.5-7B-Instruct"):
        self.client = OpenAI(
            api_key=os.getenv("FPT_API_KEY"), base_url="https://mkp-api.fptcloud.com"
        )
        self.model_name = model_name

    def complete(self, input: str) -> str:
        response = self.client.chat.completions.create(
            model=self.model_name,
            messages=[{"role": "user", "content": input}],
            max_tokens=1000,
            temperature=0.01,
        )
        return response.choices[0].message.content.strip()


class GroqLlm:
    def __init__(self, model_name: str = "gemma2-9b-it"):
        self.client = OpenAI(
            api_key=os.getenv("GROQ_API_KEY"), base_url="https://api.groq.com/openai/v1"
        )
        self.model_name = model_name

    def list_models(self):
        return self.client.models.list().data

    def complete(self, input: str) -> str:
        response = self.client.chat.completions.create(
            model=self.model_name,
            messages=[{"role": "user", "content": input}],
            max_tokens=1000,
            temperature=0.01,
        )
        return response.choices[0].message.content.strip()


llm = FPTLlm(model_name=model_name)
# llm = GroqLlm(model_name=model_name)
# pprint(llm.list_models())

In [ ]:
import json

prompt_path = "prompts_TOT.json"
# prompt_path = "prompts_vn.json"
with open(prompt_path, "r", encoding="utf-8") as file:
    prompts_template = json.load(file)
prompts_template

In [ ]:
OUTPUT_ESSAY = """
Câu trả lời nên được đặt bên trong ```answer```. 
Ví dụ tham khảo:
Ngữ cảnh: doanh nghiệp phải đền bù cho người lao động 2 tháng lương nếu doanh nghiệp đơn phương chấm dứt hợp đồng lao động.
Câu hỏi: Doanh nghiệp phải đền bù bao nhiêu tháng lương cho người lao động nếu chấm dứt hợp đồng lao động trước thời hạn?
Theo ngữ cảnh được cung cấp nếu như phải chấm dứt hợp đồng lao động trước thời hạn thì doanh nghiệp phải đền bù cho người lao động 2 tháng lương.
```answer
2 tháng.
``` 
"""
OUTPUT_OPTIONS = """
Câu trả lời (chỉ trả lời `Đúng` hoặc `Sai`) nên được đặt bên trong ```answer```.
Ví dụ tham khảo:
Ngữ cảnh: Người chưa đủ 18 tuổi không được uống rượu, bia.
Câu hỏi: Người 16 tuổi có được phép uống rượu không?
Theo ngữ cảnh được cung cấp, người 16 tuổi không được phép uống rượu.
```answer
Đúng.
```
"""
OUTPUT_CHOICES = """
Câu trả lời (chỉ trả lời `A`, `B`, `C` hoặc `D`) nên được đặt bên trong ```answer```.
Ví dụ tham khảo:
Ngữ cảnh: Người chưa đủ 18 tuổi không được uống rượu, bia.
Câu hỏi: Người 16 tuổi có được phép uống rượu không?
Theo ngữ cảnh được cung cấp, người 16 tuổi không được phép uống rượu.
```answer
C
``` 
"""

POST_PROCESS_PROMPT = """
Bạn là một hệ thống trích xuất thông tin. Nhiệm vụ của bạn là **chỉ trích xuất chính xác phần thông tin được yêu cầu từ văn bản ngữ cảnh dưới đây**.

### Yêu cầu:
- **Chỉ trích xuất thông tin trả lời cho câu hỏi hoặc đoạn đầu vào.**
- **Không thêm bất kỳ thông tin, câu văn, hay giải thích nào khác.**
- **Không lặp lại đầu vào.**
- **Trích xuất đúng phần nội dung trong văn bản, giữ nguyên cách diễn đạt.**
- Nếu không tìm thấy thông tin phù hợp, trả về **"Không tìm thấy thông tin."**

---

### Câu hỏi:

Hành vi nào được coi là cản trở việc xác minh chứng cứ?

---

### Ví dụ:

**Đầu vào:**  
Hành vi cản trở người phiên dịch thực hiện nhiệm vụ hoặc buộc người phiên dịch dịch không trung thực, không khách quan, không đúng nghĩa (điểm 6) và Hành vi cố ý dịch sai sự thật (điểm 8) và Hành vi không cử người tham gia Hội đồng định giá theo yêu cầu của Tòa án mà không có lý do chính đáng; không tham gia thực hiện nhiệm vụ của Hội đồng định giá mà không có lý do chính đáng (điểm 9) và Hành vi cản trở người tiến hành tố tụng xem xét, thẩm định tại chỗ, quyết định định giá, quyết định trưng cầu giám định hoặc xác minh, thu thập chứng cứ khác theo quy định của Luật này (điểm 7) được coi là hành vi cản trở thu thập xác minh chứng cứ của tòa án.

**Đầu ra:**  
Dịch sai sự thật

---

**Đầu vào:**  
Những vụ án hành chính không tiến hành đối thoại được là những vụ án thuộc các trường hợp sau: người khởi kiện, người bị kiện, người có quyền lợi, nghĩa vụ liên quan đã được Tòa án triệu tập hợp lệ lần thứ hai mà vẫn cố tình vắng mặt; đương sự không thể tham gia đối thoại được vì có lý do chính đáng; các bên đương sự thống nhất đề nghị không tiến hành đối thoại.

**Đầu ra:**  
Các bên đương sự đã được triệu tập lần thứ hai mà vẫn cố tình vắng mặt, hoặc không thể tham gia đối thoại, hoặc thống nhất không tiến hành đối thoại

---

**Đầu vào:**  
Theo quy định của pháp luật, nguyên tắc đối thoại trong vụ án hành chính được quy định như sau: bảo đảm công khai, dân chủ, tôn trọng ý kiến của đương sự; không được ép buộc các đương sự thực hiện việc giải quyết vụ án hành chính trái với ý chí của họ; nội dung đối thoại, kết quả đối thoại thành giữa các đương sự không trái pháp luật, trái đạo đức xã hội.

**Đầu ra:**  
Nguyên tắc đối thoại là bắt buộc

---

**Đầu vào:**  
Khi đối thoại thành thì Thẩm phán sẽ ra quyết định công nhận kết quả đối thoại và đình chỉ việc giải quyết vụ án, quyết định này có hiệu lực ngay lập tức và không bị kháng cáo hay kháng nghị theo thủ tục phúc thẩm.

**Đầu ra:**  
Quyết định có hiệu lực thi hành ngay và không bị kháng cáo, kháng nghị theo thủ tục phúc thẩm

---

**Đầu vào:**  
Trong vụ án hành chính thì không có hòa giải mà tổ chức đối thoại để các đương sự thống nhất với nhau về việc giải quyết vụ án. Trường hợp đối thoại thành thì Thẩm phán ra quyết định công nhận kết quả đối thoại thành, đình chỉ việc giải quyết vụ án.

**Đầu ra:**  
Không có hòa giải mà tổ chức đối thoại

##Câu hỏi: {question}
**Đầu vào:** 
"""

In [ ]:
example = []
added_types = set()
for sample in json_data:
    qt = sample["question_type"].lower()
    if qt in {"đúng/sai", "tự luận", "trắc nghiệm"} and qt not in added_types:
        example.append(sample)
        added_types.add(qt)
    if len(added_types) == 9:
        break
example

In [ ]:
def build_prompts(
    example,
    corpus=corpus,
    prompts_template=prompts_template,
    prompt_index=0,
    OUTPUT_OPTIONS=OUTPUT_OPTIONS,
    OUTPUT_ESSAY=OUTPUT_ESSAY,
    OUTPUT_CHOICES=OUTPUT_CHOICES,
):
    prompts = []
    for sample in example:
        qt = sample["question_type"].lower()
        premise = ""
        try:
            for article in sample["relevant_articles"]:
                law_id = article["law_id"]
                article_id = article["article_id"]

                law_corpus = next((law for law in corpus if law["id"] == law_id), None)
                if law_corpus is None:
                    raise KeyError(f"law_id '{law_id}' not found in corpus")

                article_corpus = next(
                    (a for a in law_corpus["articles"] if a["id"] == str(article_id)),
                    None,
                )
                if article_corpus is None:
                    raise KeyError(
                        f"article_id '{article_id}' not found in law_id '{law_id}'"
                    )
                premise += f"\n{article_corpus['text']}"

            if qt == "đúng/sai":
                prompt = prompts_template["truefalse"][prompt_index].format(
                    premise=premise, hypothesis=sample["text"]
                )
                prompt = OUTPUT_OPTIONS + prompt

            elif qt == "tự luận":
                prompt = prompts_template["essay"][prompt_index].format(
                    premise=premise, hypothesis=sample["text"]
                )
                prompt = OUTPUT_ESSAY + prompt

            elif qt == "trắc nghiệm":
                prompt = prompts_template["options"][prompt_index].format(
                    premise=premise,
                    hypothesis=sample["text"],
                    choices={
                        "A": sample["choices"]["A"],
                        "B": sample["choices"]["B"],
                        "C": sample["choices"]["C"],
                        "D": sample["choices"]["D"],
                    },
                )
                prompt = OUTPUT_CHOICES + prompt

            prompts.append(
                {
                    "question_id": sample["question_id"],
                    "prompt": prompt,
                    "question_type": qt,
                    "question": sample["text"],
                }
            )
        except KeyError as e:
            print(f"Error processing sample {sample.get('question_id', '')}: {e}")
            print(prompt)
    return prompts


def extract_output(output, question_type, question, llm=None, max_retries=2):
    import re

    def extract_truefalse(text):
        lowered = text.lower().replace(".", "").strip()
        if "đúng" in lowered:
            return "Đúng"
        if "sai" in lowered:
            return "Sai"
        return lowered.strip().replace(".", "")

    def extract_choices(text):
        cleaned = text.strip().replace(".", "")
        for choice in ["A", "B", "C", "D"]:
            if re.match(rf"^\s*{choice}\b", cleaned, re.IGNORECASE):
                return choice
        match = re.search(r"\b([A-D])\b", cleaned, re.IGNORECASE)
        if match:
            return match.group(1).upper()
        return cleaned.strip().replace(".", "")

    def extract_answer(text):
        text = re.sub(r"<think>.*?</think>", "", text, flags=re.DOTALL)
        # Match the specific answer format for both patterns
        match = re.search(
            r"(?:\*\*answer\*\*|```answer)\s*([\s\S]+?)\s*(?:```)?$", text, re.MULTILINE
        )
        answer = match.group(1).strip() if match else text.strip()
        return answer.replace(".", "")

    output = extract_answer(output)

    if question_type.lower() == "đúng/sai":
        return extract_truefalse(output)

    if question_type.lower() == "trắc nghiệm":
        return extract_choices(output)

    if question_type.lower() == "tự luận" and llm is not None:
        for _ in range(max_retries):
            # try to get a clean answer by prompting the LLM again for just the answer
            try:
                followup_prompt = POST_PROCESS_PROMPT.format(question=question) + output
                llm_result = llm.complete(followup_prompt)
                clean = extract_answer(llm_result)
                if clean:
                    return clean
            except Exception:
                continue
        return output.strip()

    return output.strip()


prompts = build_prompts(
    example,
    corpus,
    prompts_template,
    prompt_index=0,
    OUTPUT_OPTIONS=OUTPUT_OPTIONS,
    OUTPUT_ESSAY=OUTPUT_ESSAY,
    OUTPUT_CHOICES=OUTPUT_CHOICES,
)


In [ ]:
input = prompts[0]
response = llm.complete(input["prompt"])
print(extract_output(response, input["question_type"], input["question"]))

In [ ]:
def acc_eval_from_list(prediction_list, gold_list):
    """
    Evaluate accuracy from prediction and gold answer lists.
    Each element in both lists is a dict with keys: 'question_id' and 'answer'.
    Returns accuracy float.
    """
    gold_dict = {item["question_id"]: item["answer"] for item in gold_list}
    pred_dict = {item["question_id"]: item["answer"] for item in prediction_list}
    count_true = 0
    total = len(gold_dict)
    for qid, gold_ans in gold_dict.items():
        pred_ans = pred_dict.get(qid)
        if pred_ans is not None and pred_ans.lower() == gold_ans.lower():
            count_true += 1
    acc = count_true / total if total else 0
    print("Acc = {}".format(acc))
    return acc


In [ ]:
# Evaluation with logging and JSON output for evaluation metrics
from tqdm import tqdm
import json
import logging


eval_path = "evaluation_task2"
output_file = f"result_task_2_{model_name}"
prompt_used_index = 0
max_attempts = 3
log_path = os.path.join(eval_path, "evaluation.log")
os.makedirs(eval_path, exist_ok=True)
# Attempt to delete the log file if it exists and is not locked
try:
    if os.path.exists(log_path):
        os.remove(log_path)
except PermissionError:
    pass
# Set up logging
logging.basicConfig(
    filename=log_path,
    filemode="w",
    format="%(asctime)s %(levelname)s: %(message)s",
    level=logging.INFO,
)

prompts = build_prompts(
    json_data,
    corpus,
    prompts_template,
    prompt_index=prompt_used_index,
    OUTPUT_OPTIONS=OUTPUT_OPTIONS,
    OUTPUT_ESSAY=OUTPUT_ESSAY,
    OUTPUT_CHOICES=OUTPUT_CHOICES,
)
results = []
predictions = []

for input in tqdm(prompts):
    attempt = 0
    success = False
    log_entry = {
        "question_id": input["question_id"],
        "prompt": input["prompt"],
        "attempts": [],
        "final_answer": None,
        "error": None,
    }
    logging.info(f"Evaluating question_id={input['question_id']}")
    while attempt < max_attempts and not success:
        try:
            answer = llm.complete(input["prompt"])
            parsed_answer = extract_output(
                answer, input["question_type"], input["question"]
            )
            log_entry["attempts"].append(
                {"attempt": attempt + 1, "output": parsed_answer, "error": None}
            )
            log_entry["final_answer"] = parsed_answer
            success = True
            logging.info(
                f"Success on attempt {attempt + 1} for question_id={input['question_id']}"
            )
        except Exception as e:
            log_entry["attempts"].append(
                {"attempt": attempt + 1, "output": None, "error": str(e)}
            )
            log_entry["error"] = str(e)
            logging.error(
                f"Error on attempt {attempt + 1} for question_id={input['question_id']}: {e}"
            )
            attempt += 1
    results.append(log_entry)
    predictions.append(
        {"question_id": input["question_id"], "answer": log_entry["final_answer"]}
    )
# Save predictions and evaluation metrics in a single JSON file
acc = acc_eval_from_list(predictions, json_data)
eval_metrics = {
    "accuracy": acc,
    "total_questions": len(json_data),
    "evaluated_questions": len(predictions),
}
pairs = [
    {
        "question_id": pred["question_id"],
        "predict": pred["answer"],
        "gold": next(
            (
                item["answer"]
                for item in json_data
                if item["question_id"] == pred["question_id"]
            ),
            None,
        ),
    }
    for pred in predictions
]
output = {"evaluation": eval_metrics, "results": pairs}
with open(
    os.path.join(eval_path, "result_task_2_baseline_ToT.json"), "w", encoding="utf-8"
) as f:
    json.dump(output, f, ensure_ascii=False, indent=2)

logging.info("Evaluation completed and results written. Accuracy: {:.4f}".format(acc))


acc = acc_eval_from_list(predictions, json_data)
eval_metrics = {
    "accuracy": acc,
    "total_questions": len(json_data),
    "evaluated_questions": len(predictions),
}
pairs = [
    {
        "question_id": pred["question_id"],
        "predict": pred["answer"],
        "gold": next(
            (
                item["answer"]
                for item in json_data
                if item["question_id"] == pred["question_id"]
            ),
            None,
        ),
    }
    for pred in predictions
]
output = {"evaluation": eval_metrics, "results": pairs}
with open(os.path.join(eval_path, output_file), "w", encoding="utf-8") as f:
    json.dump(output, f, ensure_ascii=False, indent=2)

logging.info("Evaluation completed and results written. Accuracy: {:.4f}".format(acc))